In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

sns.set_theme()

In [ ]:
df = pd.read_csv("bv_benchmark.csv")
df

In [ ]:
df["n"] = df["name"].str.split("/").str[-1].astype(int)
df["T"] = df["name"].str.split("/").str[0]
df["time (ns)"] = df["cpu_time"]
df["time (ns) / n"] = df["time (ns)"] / df["n"]
df["time (ns) / log(n)"] = df["time (ns)"] / np.log(df["n"])
df["config"] = df["T"].str.split("<").str[2].str[:-2]
df["benchmark"] = df["T"].str.split("<").str[0].str[3:]
df.loc[df["config"].isnull(), "config"] = "naive"

In [ ]:
for benchmark in ("Select", "Rank", "Insert", "Remove"):
    d = df[(df["benchmark"] == benchmark) & ~(df["config"] == "naive")]

    fig, ax = plt.subplots()
    sns.lineplot(x="n", y="time (ns) / log(n)", hue="config", data=d, ax=ax)

    ax.set_xscale("log")
    ax.set_ylim((0, None))
    ax.set_title(f"BV Benchmark {benchmark}")
    plt.savefig(f"bv-{benchmark}.png")
    plt.show()

In [ ]:
for config in ("uint64_t, 1024", "uint64_t, 256", "uint64_t, 64", "uint64_t, 16", "uint8_t, 2", "naive"):
    d = df[(df["benchmark"] != "InsertN") & (df["config"] == config)]

    fig, ax = plt.subplots(figsize=(6, 4))
    ax.set_title(f"BV Benchmarks - {config}")
    sns.lineplot(x="n", y="time (ns)", hue="benchmark", data=d, ax=ax)

    if "," in config:
        T, B = config.split(", ")
        T, B = int(T[4:-2]), int(B)
        ax.axvline(T * B, c="k", alpha=0.5, ls="--")
    ax.set_xscale("log")
    ax.set_ylim((0, None))
    plt.savefig(f"bv-all-{config.replace(' ', '')}.png")
    plt.show()